# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to C:\Users\Ethan
[nltk_data]     Kuo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Ethan
[nltk_data]     Kuo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql("SELECT * FROM Messages", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """
    inputs:
    messages
       
    Returns:
    list of words into numbers of same meaning
    """
    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # normalization word tokens and remove stop words
    normlizer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    normlized = [normlizer.stem(word) for word in tokens if word not in stop_words]
    
    return normlized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipe = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

np.random.seed(13)
pipe.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000019D2ABB3CA0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def eval_metrics(ArrayL, ArrayP, col_names):
    """Evalute metrics of the ML pipeline model
    
    inputs:
    ArrayL: array. Array containing the real labels.
    ArrayP: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the ArrayP fields.
       
    Returns:
    data_metrics: Contains accuracy, precision, recall 
    and f1 score for a given set of ArrayL and ArrayP labels.
    """
    metrics = []
    
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(ArrayL[:, i], ArrayP[:, i])
        precision = precision_score(ArrayL[:, i], ArrayP[:, i])
        recall = recall_score(ArrayL[:, i], ArrayP[:, i])
        f1 = f1_score(ArrayL[:, i], ArrayP[:, i])
        
        metrics.append([accuracy, precision, recall, f1])
    
    # store metrics
    metrics = np.array(metrics)
    data_metrics = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return data_metrics    

In [7]:
# Calculate evaluation metrics for training set
Y_train_pred = pipe.predict(X_train)
col_names = list(Y.columns.values)

print(eval_metrics(np.array(Y_train), Y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.998258   0.999330  0.998393  0.998862
request                 0.999027   0.998505  0.995826  0.997164
offer                   0.999898   1.000000  0.978947  0.989362
aid_related             0.998822   0.998777  0.998411  0.998594
medical_help            0.999590   1.000000  0.994888  0.997438
medical_products        0.999846   0.998991  0.997984  0.998487
search_and_rescue       0.999898   0.998249  0.998249  0.998249
security                0.999795   0.997101  0.991354  0.994220
military                0.999846   0.996865  0.998430  0.997647
water                   1.000000   1.000000  1.000000  1.000000
food                    0.999949   0.999545  1.000000  0.999773
shelter                 0.999898   1.000000  0.998847  0.999423
clothing                0.999898   0.996753  0.996753  0.996753
money                   1.000000   1.000000  1.000000  1.000000
missing_people          1.000000   1.000

In [9]:
# Calculate evaluation metrics for test set
Y_test_pred = pipe.predict(X_test)

eval_metrics0 = eval_metrics(np.array(Y_test), Y_test_pred, col_names)
print(eval_metrics0)

                        Accuracy  Precision    Recall        F1
related                 0.820040   0.837362  0.948460  0.889455
request                 0.894575   0.835913  0.482143  0.611552
offer                   0.996465   0.000000  0.000000  0.000000
aid_related             0.772553   0.740088  0.689810  0.714065
medical_help            0.921777   0.575758  0.073218  0.129915
medical_products        0.954049   0.775000  0.096573  0.171745
search_and_rescue       0.977716   1.000000  0.052288  0.099379
security                0.980636   0.250000  0.008065  0.015625
military                0.966498   0.608696  0.062780  0.113821
water                   0.956201   0.870056  0.370192  0.519393
food                    0.936530   0.809145  0.562155  0.663407
shelter                 0.937606   0.814545  0.386874  0.524590
clothing                0.986476   0.764706  0.134021  0.228070
money                   0.979099   0.777778  0.049645  0.093333
missing_people          0.987245   0.000

In [10]:
# Calculation the proportion of each column that have label == 1
Y.sum()/len(Y)

related                   0.764792
request                   0.171892
offer                     0.004534
aid_related               0.417243
medical_help              0.080068
medical_products          0.050446
search_and_rescue         0.027816
security                  0.018096
military                  0.033041
water                     0.064239
food                      0.112302
shelter                   0.088904
clothing                  0.015560
money                     0.023206
missing_people            0.011449
refugees                  0.033618
death                     0.045874
other_aid                 0.132396
infrastructure_related    0.065506
transport                 0.046143
buildings                 0.051214
electricity               0.020440
tools                     0.006109
hospitals                 0.010873
shops                     0.004610
aid_centers               0.011872
other_infrastructure      0.044222
weather_related           0.280352
floods              

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
# Define performance metric for use in grid search scoring object
def perform_metric(Y_T, Y_P):
    """Median F1 score for all classifiers
    
    inputs:
    Y_T: array. Array containing ArrayL labels.
    Y_P: array. Array containing ArrayP labels.
        
    Routputs:
    Median F1 score for all  classifiers
    """
    f1_list = []
    for i in range(np.shape(Y_P)[1]):
        f1 = f1_score(np.array(Y_T)[:, i], Y_P[:, i])
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

In [ ]:
# grid search

parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

scorer = make_scorer(perform_metric)
cv = GridSearchCV(pipe, param_grid = parameters, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(74)
T_model = cv.fit(X_train, Y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.138, total=  40.3s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   40.2s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.139, total=  41.2s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


...

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  3.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.103, total=  43.6s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  3.7min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, score=0.176, total=  35.6s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  4.3min remaining:    0.0s


In [ ]:
#grid search results
T_model.cv_results_

In [ ]:
# top mean score
np.max(T_model.cv_results_['mean_test_score'])

In [ ]:
# Parameters for top mean score
T_model.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# evaluating metrics for test set
tuned_pred_test = T_model.predict(X_test)

eval_metrics1 = eval_metrics(np.array(Y_test), tuned_pred_test, col_names)

print(eval_metrics1)

In [ ]:
# summary of 1st model
eval_metrics0.describe()

In [ ]:
# summary of tuned model
eval_metrics1.describe()

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# using SVM instead 
pipe2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC()))
])

parameters2 = {'vect__min_df': [5],
              'tfidf__use_idf':[True],
              'clf__estimator__kernel': ['poly'], 
              'clf__estimator__degree': [1, 2, 3],
              'clf__estimator__C':[1, 10, 100]}

cv2 = GridSearchCV(pipe2, param_grid = parameters2, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(77)
T_model2 = cv2.fit(X_train, Y_train)

In [ ]:
# grid search results
T_model2.cv_results_

median always zero therefore lets evaluate metrics

In [ ]:
# Calculate evaluation metrics for test set
tuned_pred_test2 = T_model2.predict(X_test)

eval_metrics2 = eval_metrics(np.array(Y_test), tuned_pred_test2, col_names)

print(eval_metrics2)

SVM doesnt work ,we will be using RANDOM FOREST *FTW*

### 9. Export your model as a pickle file

In [ ]:
# Pickle best model
pickle.dump(T_model, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.